<a href="https://colab.research.google.com/github/martinpius/GANS/blob/main/SIMPLE%20GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>> You are on CoLaB with torch version: {torch.__version__}")
except Exception as e:
  print(f">>>> {type(e)}: {e}\n>>>> please correct {type(e)} and reload")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device('cpu')
def time_fmt(t: float = 125.39)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h} hrs: {m:>02} min: {s:>05.2f} sec"
print(f">>>> time formating\tplease wait........\n>>>> time elapsed: {time_fmt()}")


Mounted at /content/drive
>>>> You are on CoLaB with torch version: 1.8.1+cu101
>>>> time formating	please wait........
>>>> time elapsed: 0 hrs: 02 min: 05.00 sec


In [3]:
###########The simple GAN ############## Implementantion from scratch with Pytorch


In [4]:
import torch
import numpy as np
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import math, time, random
from tqdm import tqdm

In [5]:
#Setup the random seeds for reproducability.
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [6]:
#The Discriminator Network
class Discriminator(nn.Module):
  def __init__(self, in_features):
    super().__init__()
    self.discriminator = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid(),)
    
  def forward(self, input_tensor):
    return self.discriminator(input_tensor)

In [7]:
class Generator(nn.Module):
  def __init__(self, z_dim, img_dim):
    super(Generator, self).__init__()
    self.generator = nn.Sequential(
        nn.Linear(z_dim, 256),
        nn.LeakyReLU(0.01),
        nn.Linear(256, img_dim),
        nn.Tanh())
  
  def forward(self, input_tensor):
    return self.generator(input_tensor)

In [8]:
#Testing these networks if they deliver outputs of derised shapes:
def __test__():
  input_dim = 784
  batch_size = 64
  z_dim = 100
  noise = torch.randn(batch_size, z_dim)
  rand_data = torch.randn(batch_size, input_dim)
  disc = Discriminator(input_dim)
  gen = Generator(z_dim, input_dim)
  gen_out = gen(noise)
  disc_out = disc(rand_data)
  return f" gen_out_shape: {gen_out.shape} disc_out_shape: {disc_out.shape}" 

In [9]:
__test__()

' gen_out_shape: torch.Size([64, 784]) disc_out_shape: torch.Size([64, 1])'

In [10]:
#Get and preprocess from torchvision:
batch_size = 32
transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])

dfm = datasets.MNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dfm, batch_size=batch_size, shuffle=True)


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw




Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw




Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw




Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [11]:
#The training loop:
EPOCHS = 50
learning_rate = 3e-4
z_dim = 64
img_dim = 784
batch_size = batch_size
fixed_noise = torch.randn(batch_size, z_dim).to(device = device)
generator = Generator(z_dim, img_dim).to(device = device)
discriminator = Discriminator(img_dim).to(device = device)
real_writer = SummaryWriter(f"runs/mnist_images/real")
fake_writer = SummaryWriter(f"runs/mnist_images/fake")


In [12]:
loss = nn.BCELoss()
disc_opt = optim.Adam(params = discriminator.parameters(), lr = learning_rate)
gen_opt = optim.Adam(params = generator.parameters(), lr = learning_rate)


In [ ]:
step = 0
global_tic = time.time()
for epoch in range(EPOCHS):
  tic = time.time()
  print(f">>>> training starts for epoch {epoch + 1}\tplease wait...............")
  for idx, (real, _) in enumerate(tqdm(loader)):
    real = real.view(-1, 784).to(device)
    batch_size = real.shape[0]
    noise = torch.randn(batch_size, z_dim).to(device)
    fake = generator(noise)
    disc_real = discriminator(real).view(-1)
    lossD_real = loss(disc_real, torch.ones_like(disc_real))
    disc_fake = discriminator(fake).view(-1)
    lossD_fake = loss(disc_fake, torch.zeros_like(disc_fake))
    lossD = (lossD_real + lossD_fake) / 2
    discriminator.zero_grad()
    lossD.backward(retain_graph=True)
    disc_opt.step()

    #training the generator:
    gen_out = discriminator(fake)
    gen_loss = loss(gen_out, torch.ones_like(gen_out))
    generator.zero_grad()
    gen_loss.backward()
    gen_opt.step()
    toc = time.time()
    if idx % 200 == 0:
      print(f"\n>>>> time elapsed at the end of epoch {epoch + 1} of batch {idx} is {time_fmt(toc - tic)}")
      print(f">>>> generator loss: {gen_loss:.4f} | generator PPL: {math.exp(gen_loss):7.4f}")
      print(f">>>> discriminator loss: {lossD:.4f} | discriminator PPL: {math.exp(lossD):7.4f}")
    with torch.no_grad():
      fake_images = generator(fixed_noise).reshape(-1, 1, 28, 28)
      real_images = real.reshape(-1, 1, 28, 28)
      img_grid_real = torchvision.utils.make_grid(real_images, normalize = True)
      img_grid_fake = torchvision.utils.make_grid(fake_images, normalize = True)
      real_writer.add_image('real_mnist_image', img_grid_real, global_step = step)
      fake_writer.add_image('fake_mnist_image', img_grid_fake, global_step = step)
      step+=1
global_toc = time.time()
print(f"\n>>>> total time elapsed for trainining this simple GAN is {time_fmt(global_toc - global_tic)}")






  0%|          | 0/1875 [00:00<?, ?it/s]

>>>> training starts for epoch 1	please wait...............

>>>> time elapsed at the end of epoch 1 of batch 0 is 0 hrs: 00 min: 00.00 sec
>>>> generator loss: 0.7300 | generator PPL:  2.0752
>>>> discriminator loss: 0.6582 | discriminator PPL:  1.9314


 11%|█         | 205/1875 [00:08<01:05, 25.62it/s]


>>>> time elapsed at the end of epoch 1 of batch 200 is 0 hrs: 00 min: 08.00 sec
>>>> generator loss: 2.3337 | generator PPL: 10.3161
>>>> discriminator loss: 0.1143 | discriminator PPL:  1.1211


 21%|██▏       | 403/1875 [00:15<00:55, 26.69it/s]

...


>>>> time elapsed at the end of epoch 16 of batch 1200 is 0 hrs: 00 min: 37.00 sec
>>>> generator loss: 2.1928 | generator PPL:  8.9602
>>>> discriminator loss: 0.4537 | discriminator PPL:  1.5741


 75%|███████▍  | 1404/1875 [00:43<00:14, 31.70it/s]


>>>> time elapsed at the end of epoch 16 of batch 1400 is 0 hrs: 00 min: 43.00 sec
>>>> generator loss: 1.3876 | generator PPL:  4.0054
>>>> discriminator loss: 0.3760 | discriminator PPL:  1.4565


 85%|████████▌ | 1603/1875 [00:50<00:09, 30.20it/s]


>>>> time elapsed at the end of epoch 16 of batch 1600 is 0 hrs: 00 min: 50.00 sec
>>>> generator loss: 1.5308 | generator PPL:  4.6218
>>>> discriminator loss: 0.6945 | discriminator PPL:  2.0028
